In [103]:
import os, sys
sys.path.append("../../../")

from src.core.module import Module, Linear, LayerNorm
from src.core.losses import CrossEntropy, BCE
from src.core.optim import Standard, AdamW
from src.core.tensor import Tensor
from src.utils.lr_scheduler import LRScheduler
import numpy as np
import time
from typing import List
# from src.tokenizer.tokenizer import Tokenizer
import pandas as pd

In [104]:
class Net(Module):
    def __init__(self):
        super().__init__()
        self.fc1 = self.linear(7, 28, name="fc1")
        self.fc2 = self.linear(28, 2, name="fc2")
        self.ln = self.layer_norm(axis=-1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.ln(x)
        x = self.gelu(x)
        x = self.fc2(x)
        return x
    
    def train(self, x: Tensor, y: Tensor, optimizer, num_epochs=100):
        for epoch in range(num_epochs):
            y_hat = self.forward(x)
            
            loss = CrossEntropy(y_hat, y)
            
            loss.backward()
            
            optimizer.step()
            optimizer.zero_grad()
            if epoch % 10 == 0:

                print(f"Epoch {epoch}, Loss: {loss.data}")





df = pd.read_csv("../../../src/experiments/data.csv")
df = df.sample(frac=1).reset_index(drop=True)
df['Quality'] = df['Quality'].apply(lambda x: 1 if x == "Good" else 0)
X = Tensor(np.array(df.drop('Quality', axis=1).values))[:128]
y = Tensor(np.array(df['Quality'].values))[:128] 

X_test = Tensor(np.array(df.drop('Quality', axis=1).values))[128:]
y_test = Tensor(np.array(df['Quality'].values)).reshape((-1, 1))[128:]
net = Net()

net._build(X.shape)
optimizer = AdamW(net.parameters(), lr=0.001, clip_norm=100.0)

net.train(X, y, optimizer, num_epochs=250)


Epoch 0, Loss: 0.6282037930906555
Epoch 10, Loss: 0.5422915212946702
Epoch 20, Loss: 0.4699268396060313
Epoch 30, Loss: 0.4121579416730207
Epoch 40, Loss: 0.36552092102756795
Epoch 50, Loss: 0.327235274203743
Epoch 60, Loss: 0.295288346741716
Epoch 70, Loss: 0.268268900554893
Epoch 80, Loss: 0.24510937118890805
Epoch 90, Loss: 0.22501613207572024
Epoch 100, Loss: 0.207412333049481
Epoch 110, Loss: 0.1918794298023724
Epoch 120, Loss: 0.1780966995945082
Epoch 130, Loss: 0.16580329221929324
Epoch 140, Loss: 0.15477972907221776
Epoch 150, Loss: 0.14483713570819368
Epoch 160, Loss: 0.13581230763971272
Epoch 170, Loss: 0.1275663562007921
Epoch 180, Loss: 0.11998408711198164
Epoch 190, Loss: 0.11297306816577213
Epoch 200, Loss: 0.10646179073939649
Epoch 210, Loss: 0.10039659383233554
Epoch 220, Loss: 0.09473771998806726
Epoch 230, Loss: 0.0894553280957046
Epoch 240, Loss: 0.08452571908872158


In [101]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd

# Define the model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(7, 28)
        self.fc4 = nn.Linear(28, 3)  # 3 classes
        self.gelu = nn.GELU()

    def forward(self, x):
        x = self.fc1(x)
        x = self.gelu(x)
        x = self.fc4(x)
        return x  # raw logits, no sigmoid

# Load + prep data
df = pd.read_csv("../../../src/experiments/data.csv")
df = df.sample(frac=1).reset_index(drop=True)

# Map classes properly starting at 0
df['Quality'] = df['Quality'].apply(lambda x: 0 if x == "Good" else (1 if x == "Bad" else 2))  # Adjust if you have 3 classes

X = torch.tensor(df.drop('Quality', axis=1).values, dtype=torch.float32)
y = torch.tensor(df['Quality'].values, dtype=torch.long)  # LONG for CE loss!

# Train/test split
X_train, y_train = X[:128], y[:128]
X_test, y_test = X[128:], y[128:]

# Initialize model, loss, optimizer
model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001)

# Training loop
for epoch in range(250):
    model.train()

    y_pred = model(X_train)
    loss = criterion(y_pred, y_train)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")


Epoch 0, Loss: 1.3029
Epoch 10, Loss: 1.1438
Epoch 20, Loss: 1.0108
Epoch 30, Loss: 0.8985
Epoch 40, Loss: 0.8017
Epoch 50, Loss: 0.7168
Epoch 60, Loss: 0.6414
Epoch 70, Loss: 0.5736
Epoch 80, Loss: 0.5124
Epoch 90, Loss: 0.4572
Epoch 100, Loss: 0.4080
Epoch 110, Loss: 0.3646
Epoch 120, Loss: 0.3270
Epoch 130, Loss: 0.2947
Epoch 140, Loss: 0.2674
Epoch 150, Loss: 0.2442
Epoch 160, Loss: 0.2246
Epoch 170, Loss: 0.2080
Epoch 180, Loss: 0.1938
Epoch 190, Loss: 0.1817
Epoch 200, Loss: 0.1711
Epoch 210, Loss: 0.1617
Epoch 220, Loss: 0.1534
Epoch 230, Loss: 0.1459
Epoch 240, Loss: 0.1392
